Dans ce notebook nous importons les données de la Banque Mondiale avec l'API

Installation des bibliothèques requises

In [ ]:
%pip install requests
%pip install Faostat

     ---------------------------------------- 64.9/64.9 kB 3.6 MB/s eta 0:00:00
     -------------------------------------- 102.2/102.2 kB 3.0 MB/s eta 0:00:00
     -------------------------------------- 126.3/126.3 kB 2.5 MB/s eta 0:00:00
     -------------------------------------- 167.3/167.3 kB 3.4 MB/s eta 0:00:00
     ---------------------------------------- 70.4/70.4 kB 1.9 MB/s eta 0:00:00



[notice] A new release of pip available: 22.2.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 11.6/11.6 MB 4.7 MB/s eta 0:00:00
     ------------------------------------- 508.0/508.0 kB 10.6 MB/s eta 0:00:00
     ------------------------------------- 346.6/346.6 kB 21.0 MB/s eta 0:00:00



[notice] A new release of pip available: 22.2.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Import des données économiques avec l'API de la Banque Mondiale (voir la doc sur https://datatopics.worldbank.org/world-development-indicators/)

Avant d'importer les variables dont nous avons besoin, il faut justement déterminer quelles sont ces variables. Une petite revue de littérature s'impose. 

idée: utiliser la variable "part de l'agriculture dans le PIB", "revenus de la déforestation" (si ça existe).
Référence qui revient souvent: la courbe de kuznets environnementale

Articles : 
- Andrée, B. P. J., Chamorro, A., Spencer, P., Koomen, E., & Dogo, H. (2019). Revisiting the relation between economic growth and the environment; a global assessment of deforestation, pollution and carbon emission. Renewable and Sustainable Energy Reviews, 114, 109221.)
  - inverted U-shaped relations between per capita income and deforestation
  - Poverty and income inequality correlate with higher deforestation
  - agricultural GDP shares correlate with deforestation
- López, R., & Galinato, G. I. (2005). Trade policies, economic growth, and the direct causes of deforestation. Land economics, 81(2), 145-169.
  -  Economy-wide factors such as trade openness and economic growth explain an important portion of the variation in three key factors of deforestation: poverty, agricultural expansion, and road building. 



Variables récupérées:

- Indice de GINI : SI.POV.GINI
- agricultural land (% of land area) : AG.LND.AGRI.ZS
- forest area (% of land area): AG.LND.FRST.ZS
- GDP (US$): NY.GDP.MKTP.CD
- GDP GROWTH : NY.GDP.MKTP.KD.GZ


Nous allons récupérer les données de la Banque Mondiale de deux manières différentes : la première avec un seul indicateur (au choix) pour cibler, la seconde avec tous les indicateurs en même temps pour avoir une vue d'ensemble de ce qui est disponible. 

In [6]:
import requests
import pandas as pd

# URL de base de l'API
base_url = "https://api.worldbank.org/v2/country/all/indicator/NY.GDP.MKTP.CD?date=1960:2024&format=json"

# Initialisation des variables
all_records = []
page = 1

# Requête initiale pour obtenir le nombre total de pages
url = f"{base_url}&page={page}"
req = requests.get(url)
data_eco = req.json()

# Vérification de la structure de la réponse et extraction du nombre total de pages
if len(data_eco) > 0 and isinstance(data_eco[0], dict) and "pages" in data_eco[0]:
    total_pages = data_eco[0]["pages"]
    print(f"Nombre total de pages : {total_pages}")
else:
    print("Impossible de déterminer le nombre de pages.")
    exit()

# Récupérer les données pour toutes les pages
while page <= total_pages:
    url = f"{base_url}&page={page}"
    req = requests.get(url)
    data_eco = req.json()

    # Vérification si des données existent pour cette page
    if len(data_eco) > 1 and isinstance(data_eco[1], list):
        all_records.extend(data_eco[1])  # Ajouter les données de la page actuelle
    page += 1  # Passer à la page suivante

# Transformation des données en DataFrame pandas
df = pd.DataFrame(all_records)

# Vérification et affichage des colonnes pertinentes
if "country" in df.columns and "date" in df.columns and "value" in df.columns:
    table = df[["date", "value", "country"]]
    print(table.head())  # Afficher les premières lignes du tableau
else:
    print("Les colonnes attendues ne sont pas disponibles dans les données récupérées.")


Nombre total de pages : 341
   date         value                                            country
0  2023  1.236163e+12  {'id': 'ZH', 'value': 'Africa Eastern and Sout...
1  2022  1.183962e+12  {'id': 'ZH', 'value': 'Africa Eastern and Sout...
2  2021  1.086772e+12  {'id': 'ZH', 'value': 'Africa Eastern and Sout...
3  2020  9.290741e+11  {'id': 'ZH', 'value': 'Africa Eastern and Sout...
4  2019  1.006527e+12  {'id': 'ZH', 'value': 'Africa Eastern and Sout...


In [5]:
df

,indicator,country,countryiso3code,date,value,unit,obs_status,decimal
0,"{'id': 'NY.GDP.MKTP.CD', 'value': 'GDP (curren...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2020,9.290741e+11,,,0
1,"{'id': 'NY.GDP.MKTP.CD', 'value': 'GDP (curren...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2019,1.006527e+12,,,0
2,"{'id': 'NY.GDP.MKTP.CD', 'value': 'GDP (curren...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2018,1.012719e+12,,,0
3,"{'id': 'NY.GDP.MKTP.CD', 'value': 'GDP (curren...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2017,9.401055e+11,,,0
4,"{'id': 'NY.GDP.MKTP.CD', 'value': 'GDP (curren...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2016,8.298300e+11,,,0
...,...,...,...,...,...,...,...,...
2921,"{'id': 'NY.GDP.MKTP.CD', 'value': 'GDP (curren...","{'id': 'ZW', 'value': 'Zimbabwe'}",ZWE,2014,1.949552e+10,,,0
2922,"{'id': 'NY.GDP.MKTP.CD', 'value': 'GDP (curren...","{'id': 'ZW', 'value': 'Zimbabwe'}",ZWE,2013,1.909102e+10,,,0
2923,"{'id': 'NY.GDP.MKTP.CD', 'value': 'GDP (curren...","{'id': 'ZW', 'value': 'Zimbabwe'}",ZWE,2012,1.711485e+10,,,0
2924,"{'id': 'NY.GDP.MKTP.CD', 'value': 'GDP (curren...","{'id': 'ZW', 'value': 'Zimbabwe'}",ZWE,2011,1.410192e+10,,,0


In [30]:
import requests
import pandas as pd

def fetch_worldbank_data(indicator, start_date="1960", end_date="2024"):
    """
    Récupère les données de la Banque Mondiale pour un indicateur donné.
    
    Args:
        indicator (str): Code de l'indicateur (ex. "NY.GDP.MKTP.CD").
        start_date (str): Année de début des données (par défaut "1960").
        end_date (str): Année de fin des données (par défaut "2024").
    
    Returns:
        pd.DataFrame: Tableau contenant les données récupérées, ou None en cas d'échec.
    """
    base_url = f"https://api.worldbank.org/v2/country/all/indicator/{indicator}?date={start_date}:{end_date}&format=json"
    all_records = []
    page = 1

    url = f"{base_url}&page={page}"
    req = requests.get(url)
    data_eco = req.json()

    if len(data_eco) > 0 and isinstance(data_eco[0], dict) and "pages" in data_eco[0]:
        total_pages = data_eco[0]["pages"]
        print(f"{indicator} - Nombre total de pages : {total_pages}")
    else:
        print(f"{indicator} - Impossible de déterminer le nombre de pages.")
        return None

    while page <= total_pages:
        url = f"{base_url}&page={page}"
        req = requests.get(url)
        data_eco = req.json()

        if len(data_eco) > 1 and isinstance(data_eco[1], list):
            all_records.extend(data_eco[1])
        page += 1

    df = pd.DataFrame(all_records)

    if "country" in df.columns and "date" in df.columns and "value" in df.columns:
        # Extraire le nom du pays depuis le dictionnaire dans la colonne 'country'
        df["country"] = df["country"].apply(lambda x: x["value"] if isinstance(x, dict) else x)
        table = df[["date", "value", "country"]]
        return table
    else:
        print(f"{indicator} - Les colonnes attendues ne sont pas disponibles.")
        return None

# Liste des indicateurs
indicators = {
    "NY.GDP.MKTP.CD": "PIB",
    "SI.POV.GINI": "GINI",
    "AG.LND.AGRI.ZS": "Part terres agricoles",
    "AG.LND.FRST.ZS": "Part terres foret",
}

# Récupérer les données pour chaque indicateur
data_frames = []
for indicator, name in indicators.items():
    data = fetch_worldbank_data(indicator, start_date="1960", end_date="2024")
    if data is not None:
        data["indicator"] = name  # Ajouter une colonne pour l'indicateur
        data_frames.append(data)

# Combiner toutes les données dans une table unique
merged_data = None
for df in data_frames:
    # Transformer la colonne 'value' pour avoir une colonne par indicateur
    indicator_name = df["indicator"].iloc[0]
    df = df.rename(columns={"value": indicator_name})
    df = df.drop(columns=["indicator"])
    if merged_data is None:
        merged_data = df
    else:
        merged_data = pd.merge(merged_data, df, on=["date", "country"], how="outer")

# Afficher le tableau final
print(merged_data.head())


NY.GDP.MKTP.CD - Nombre total de pages : 341
SI.POV.GINI - Nombre total de pages : 341
AG.LND.AGRI.ZS - Nombre total de pages : 341
AG.LND.FRST.ZS - Nombre total de pages : 341
   date           PIB                      country  GINI  \
0  1960           NaN                  Afghanistan   NaN   
1  1960  2.121696e+10  Africa Eastern and Southern   NaN   
2  1960  1.188413e+10   Africa Western and Central   NaN   
3  1960           NaN                      Albania   NaN   
4  1960  2.723615e+09                      Algeria   NaN   

   Part terres agricoles  Part terres foret  
0                    NaN                NaN  
1                    NaN                NaN  
2                    NaN                NaN  
3                    NaN                NaN  
4                    NaN                NaN  


In [29]:
merged_data

,date,PIB,country,GINI,Part terres agricoles,Part terres foret
0,2010,1.585667e+10,Afghanistan,NaN,58.134400,1.852782
1,2010,8.494097e+11,Africa Eastern and Southern,NaN,45.629133,32.583415
2,2010,5.986977e+11,Africa Western and Central,NaN,38.756043,20.674012
3,2010,1.192693e+10,Albania,NaN,43.843066,28.542701
4,2010,1.777851e+11,Algeria,NaN,17.371334,0.805294
...,...,...,...,...,...,...
2921,2020,1.553170e+10,West Bank and Gaza,NaN,67.751429,1.684385
2922,2020,8.557772e+13,World,NaN,36.730920,31.176905
2923,2020,NaN,"Yemen, Rep.",NaN,44.419190,1.039832
2924,2020,1.813776e+10,Zambia,NaN,32.067959,60.283337
